# Twitter Data Handeler

### 1. Data Importing
The dataset would be imported here.

NOTE:: The dataset is for non comercial purpose and was downloaded from [here](https://ieee-dataport.org/open-access/coronavirus-covid-19-tweets-dataset). I own no rights to this dataset and am not responsible for any missuse of it.

In [1]:
import types
import json
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_2bfc56ae673c4e3aad4c3da569258436 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XXX',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

def fetcher(number):
    global client_2bfc56ae673c4e3aad4c3da569258436
    body = client_2bfc56ae673c4e3aad4c3da569258436.get_object(Bucket='sentimentanalysisproject-donotdelete-pr-pkfzekostvak36',Key='corona_tweets_' + str(number).zfill(2) + '.csv')['Body']
    # add missing __iter__ method, so pandas accepts body as file-like object
    if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

    data = pd.read_csv(body, names=['tweet_id', 'sentiment_score'], )
    
    return data

data = pd.concat([fetcher(num) for num in range(1, 4)], ignore_index=True)
data.head()

ModuleNotFoundError: No module named 'botocore'

### 2. Text Hydration and Filtering

In [ ]:
!pip install twarc    #A tool used to hydrate twitter text from tweet_id

In [ ]:
def isIndian(loc):#checks id location is Indian
    try:
        if (loc == 'IN') or (loc[-5:] == 'India'):
            return True
    except:
        pass
    return False

In [ ]:
def carveData(l, tweet, ss):#Will be used to carve the required data from a tweet
    location = None
    try:
        location = tweet['user']['location']
    except:
        pass
    if not location:
        try:
            location = tweet['place']['full_name']
        except:
            pass
    if not location:
        try:
            location = tweet['retweeted_status']['user']['location']
        except:
            pass
    if not location:
        try:
            location = tweet['retweeted_status']['place']['full_name']
        except:
            pass
    if not location:
        try:
            location = tweet['place']['country_code']
        except:
            pass
    if not location:
        try:
            location = tweet['retweeted_status']['place']['country_code']
        except:
            pass
    if not location:
        return location
    if not isIndian(location):
        return
    hashtags = []
    try:
        for ht in tweet['entities']['hashtags']:
            hashtags.append(ht['text'])
    except:
        pass
    try:
        for ht in tweet['retweeted_status']['entities']['hashtags']:
            hashtags.append(ht['text'])
    except:
        pass
    user_mentions = []
    try:
        for n in tweet['entities']['user_mentions']:
            user_mentions.append(n['name'])
    except:
        pass
    try:
        for n in tweet['retweeted_status']['entities']['user_mentions']:
            user_mentions.append(n['name'])
    except:
        pass
    try:
        user_mentions.append(tweet['retweeted_status']['user']['name'])
    except:
        pass
    text = None
    try:
        text = tweet['text']
    except:
        pass
    created_at = None
    try:
        created_at = tweet['created_at']
    except:
        pass
    _id = None
    try:
        _id = tweet['id']
    except:
        pass
    k = {'id': _id, 'sentiment_score': ss, 'created_at': created_at, 'user_mentions': user_mentions, 'hashtags': hashtags, 'location': location, 'text': text}
    l.append(k)
    return _id
    

In [ ]:
from twarc import Twarc

consumer_key='XXX'
consumer_secret='XXX'
access_token='XXX'
access_token_secret='XXX'

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

#list of important Factors
#l[0]['user']['location'][-5:] == 'India'
#l[0]['entities']['hashtags']
#l[0]['place']['country_code'] == 'IN'
#l[0]['place']['full_name']
#l[0]['retweeted_status']
#for x in l[0]['entities']['user_mentions']:
#    x['name']
#l[0]['created_at']

In [ ]:
l =[]
i = 0
for index, row in data.iterrows():
    for tw in t.hydrate([int(row['tweet_id'])]):
        tweet_id = carveData(l, tw, row['sentiment_score'])
        i+=1
    print('\rObjects :: ' + str(len(l)) + '/' + str(i), end='')

In [ ]:
new_df = pd.DataFrame(l, columns=['id', 'sentiment_score', 'created_at', 'user_mentions', 'hashtags', 'location'])

In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='XXX', project_access_token='XXX')
pc = project.project_context

In [ ]:
project.save_data(data=new_df.to_csv(),file_name='processed_data_01_03.csv',overwrite=True)

### 3. Data Categorization
Here the dataset would be divided into various parts based on the timeline of actions taken by the Indian gov. The categories are listed below:
   1. Before Janata Lockdown
   2. Durring Janata Lockdown (22 March from 7 a.m. to 9 p.m.)
   3. Phase 1 (25 March – 14 April)
   4. Phase 2 (15 April – 3 May)
   5. Phase 3 (4–17 May)
   6. Phase 4 (18–31 May)
   7. Phase 5 / Unlock 1 (1–30 June) 

[-1,0) is considered negative sentiment, 0 is considered neutral and (0,1]

In [ ]:
[tw for tw in t.hydrate([1240861450941247489])]

In [ ]:
type(1240861450941247489)

In [ ]:
for x, d in zip(l, data):
    print(d[0], d[1])

In [ ]:
data = [(1240861450941247489, 1)]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data, columns=['tweet_id', 'ss'])

In [ ]:
l = range(2, 10)

In [ ]:
isIndian(tw['user']['location'])

In [ ]:
new_df

In [ ]:

data = pd.DataFrame([], columns=['tweet_id', 'sentiment_score'])

In [ ]:
data.count()

In [ ]:
for index, row in data.iterrows():
    print(int(row['tweet_id']), row['sentiment_score'])
    break

In [ ]:
l